# Домашнее задание №1
Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции.
По образу практики, попробуйте создать искусственный датасет с лишними столбцами.
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель.
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли.
Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы.
Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [73]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Для примера рассмотрим придуманную формулу Расчет базового расхода калорий в день, создадим искусственный датасет, где будут следующие столбцы:
- **weight:** вес, кг
- **growth:** рост, см
- **age:** возраст, лет
- **sleep_hours:** среднее количество часов сна в день
- **bmr:** количество калорий необходимых для жизни в состоянии покоя (питание, рост, переваривание пищи) расчитывается по формуле (0.2 * вес, кг * рост, см) — (5 * возраст) — 161, ккал/сут

In [74]:
dataset_size = 1000 #размер датасета

weight = np.random.randint(30, 120, n_samples)
growth = np.random.randint(100, 200, n_samples)
age = np.random.randint(10, 100, n_samples)
sleep_hours = np.random.randint(2, 24, n_samples)

bmr = 0.2 * weight * growth - 5 * age - 161

data = pd.DataFrame({'weight': weight, 'growth': growth, 'age': age, 'sleep_hours': sleep_hours, 'bmr': bmr})
data.head()

,weight,growth,age,sleep_hours,bmr
0,95,197,39,11,3387.0
1,100,123,98,20,1809.0
2,36,157,81,3,564.4
3,108,152,80,21,2722.2
4,117,145,32,8,3072.0


По итоговой формуле видно, что столбец sleep_hours - лишняя переменная.

Далее построим модель линейной регрессии, разделим наш датасет на x и y, в y будет bmr, в x - все остальные показатели.

In [75]:
from sklearn.metrics import mean_absolute_error

X = data[['weight', 'growth', 'age', 'sleep_hours']]
y = data['bmr']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['weight', 'growth', 'age', 'sleep_hours']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [30.11909716 15.1890225  -5.25801884  1.15648172]
Bias: -2451.0787980827245
Error: 108.16993589410477


In [76]:
y.median()

1807.3

По весам модели видно, что вес надо умножитьна 30, рост на 15, возраст на -5, а среднее кол-во часов сна на 1, а сдвиг равен -2451, а по нашей формуле он равен -161. По абсолютной ошибке видно, что предсказанные значения отклоняются от реальных на 108, с учетом,что медиана по y равно 1807, это достаточно большое значение ошибки. Попробуем убрать из x лишнюю переменную среднее кол-во часов сна.

In [77]:
X = data[['weight', 'growth', 'age']]
y = data['bmr']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['weight', 'growth', 'age']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [30.12794884 15.1848554  -5.25123113]
Bias: -2436.79922920178
Error: 108.21054671465039


После того, как мы убрали среднее количество часов сна качество модели не улучшилось и модель не может нормально предсказать bmr. Проблема в том, что формуле у нас мультипликативная модель (показатель веса умножается на показатель роста), а линейная модель - аддитивная (сумма показателей), создадим новую переменную в которой посчитаем произведение роста на вес и запустим модель линейной регрессии снова.

In [78]:
# Создаем новый признак
data['mult'] = data['weight'] * data['growth']
data.head(5)

,weight,growth,age,sleep_hours,bmr,mult
0,95,197,39,11,3387.0,18715
1,100,123,98,20,1809.0,12300
2,36,157,81,3,564.4,5652
3,108,152,80,21,2722.2,16416
4,117,145,32,8,3072.0,16965


In [79]:
X = data[['mult', 'age']]
y = data['bmr']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult', 'age']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.2 -5. ]
Bias: -161.00000000000068
Error: 3.7414338294183835e-13


Как видим, веса (коэффициенты) получились 0.2 и -5, сдвиг равен -161, а абсолютная ошибка близка к 0. Модель линейной регрессии идеально предсказала и подобрала коэффциенты, которые мы закладывали в изначальную формулу.